# Magnetotellurics

This object can be used to store magnetotelluric (MT) data - a natural-source geophysical method widely used for deep crustal investigations. Electromagnetic data are provided in the frequency-domain as point source measurments of either impedances or apparent resistity/phase.

The following example shows how to generate an MT survey with associated data programmatically.

![mtSurvey](./images/mt_survey.png)

In [ ]:
import numpy as np
from geoh5py.workspace import Workspace
from geoh5py.objects import MTReceivers

# Create a new project
workspace = Workspace("my_project.geoh5")

# Define the receiver locations on 2 lines, 60 m apart
x_loc, y_loc = np.meshgrid(np.linspace(-5, 5, 2), np.linspace(0., 20., 9))
vertices = np.c_[x_loc.ravel(), y_loc.ravel(), np.zeros_like(x_loc).ravel()]

# Create the survey from vertices
mt_survey = MTReceivers.create(workspace, vertices=vertices)

## Metadata

Along with the survey object itself, the metadata contains all the necessary information to define the geophysical experiment.

In [ ]:
mt_survey.metadata

### Channels

List of frequencies at which the data are provided.

In [ ]:
mt_survey.channels = [1., 10., 100.]

### Input type

Generic label used in the `geoh5` standard for all EM survey entities. Restricted to `Rx only` in the case of [MTReceivers](../api/geoh5py.objects.surveys.electromagnetics.rst#module-geoh5py.objects.surveys.electromagnetics.magnetotellurics) (natural sources method).

### Property groups

List of [PropertyGroup](../api/geoh5py.groups.rst#module-geoh5py.groups.property_group) defining the various data components (e.g. `Zxx (real)`, `Zxy (imag)`, ...). It is not required to supply all components of the impedence tensor, but it is expected that each component contains data channels for all frequencies and in the same order as defined in `Channels`.

The class method [add_component_data](../api/geoh5py.objects.surveys.electromagnetics.rst#geoh5py.objects.surveys.electromagnetics.base.BaseEMSurvey.add_component_data) can help users add data from nested dictionaries. Below is an example using four components:

In [ ]:
# Arbitrary data generator using sine functions
data_fun = lambda c, f: (c+1.) * np.sin(f * np.pi * (x_loc * y_loc).ravel() / 200.)

# Create a nested dictionary of component and frequency data.
data = {
    component : {
        freq: {"values": data_fun(cc, ff)} for ff, freq in enumerate(mt_survey.channels)
    } for cc, component in enumerate(["Zxx (real)", "Zxx (imaginary)", "Zxy (real)", "Zyy (real)"])
}
    
mt_survey.add_component_data(data)

Metadata are also updated to reflect the addition of component data.

In [ ]:
mt_survey.metadata

### Receivers

Generic label used in the `geoh5` standard for EM survey to identify the receiver entity. Restricted to itself in the case of `MTReceivers`.

### Unit

Units for the sampling frequency chosen from either `Hertz (Hz)`, `KiloHertz (kHz)`, `MegaHertz (MHz)` or `Gigahertz (GHz)`.